In [5]:
import requests
import shared.firestore as fs_module
from processors import (
  form_100,
  form_110,
  form_120,
  form_150,
  form_170,
  form_180,
  form_190,
  form_200,
  form_220,
  form_300,
  form_400,
  form_500,
  form_600
)

In [6]:
class MockFirestoreClient:
  def collection(self, name):
    print(f"   [DB MOCK] Accediendo a colección: '{name}'")
    return self
  
  def document(self, name):
    print(f"   [DB MOCK] Accediendo a documento: '{name}'")
    return self

  def get(self):
    print("   [DB MOCK] Simulando .get()")
    return self
  
  def set(self, data, merge=True):
    print(f"   [DB MOCK] Simulando .set() con data: {data}")

  @property
  def exists(self):
    return True

  def to_dict(self):
    return {"departamento": "COCHABAMBA (MOCK)", "nombre": "ENTIDAD MOCK"}

def mock_insert_entidad(db, cod, nombre, fax, telefono):
  print(f"💾 [GUARDAR ENTIDAD] Cod: {cod} | Nom: {nombre}")

def mock_insert_convocatoria(db, **kwargs):
  print(f"💾 [GUARDAR CONVOCATORIA] CUCE: {kwargs.get('cuce')} | Estado: {kwargs.get('estado')}")

def mock_insert_item(db, item, cuce, slug):
  desc = item.get('descripcion', '')[:30]
  print(f"📦 [GUARDAR ITEM] Slug: {slug} | Desc: {desc}... | $ {item.get('precio_unitario') or item.get('precio_adjudicado')}")

def mock_update_item(db, item_id, data):
  print(f"🔄 [ACTUALIZAR ITEM] ID: {item_id} | Data: {data}")

def mock_insert_proponente(db, nombre):
  print(f"👤 [GUARDAR PROPONENTE] {nombre}")

def mock_update_status(db, cuce, estado, form):
  print(f"🚩 [UPDATE STATUS] {cuce} -> {estado} ({form})")

# Simulamos la búsqueda de items para el Form 500
def mock_get_items_by_cuce(db, cuce):
  print(f"🔍 [GET ITEMS] Buscando items para {cuce}...")
  
  # Clase auxiliar para simular un DocumentSnapshot de Firestore
  class MockDoc:
    def __init__(self, id, data):
      self.id = id
      self._data = data
    def to_dict(self):
      return self._data
      
  # Devolvemos una lista falsa de items para que el Form 500 tenga algo que comparar
  # TIP: Ajusta la descripción aquí si quieres probar el matching exacto
  return [
    MockDoc("item_mock_1", {"descripcion": "Papel bond tamaño carta", "estado": "Solicitado", "cantidad": 100}),
    MockDoc("item_mock_2", {"descripcion": "Toner para impresora HP", "estado": "Solicitado", "cantidad": 5})
  ]

fs_module.insert_entidad = mock_insert_entidad
fs_module.insert_convocatoria = mock_insert_convocatoria
fs_module.insert_item = mock_insert_item
fs_module.update_item_adjudicacion = mock_update_item
fs_module.insert_proponente = mock_insert_proponente
fs_module.update_convocatoria_status = mock_update_status
fs_module.get_items_by_cuce = mock_get_items_by_cuce

def run_local_test(content, file_path):
  print(f"\n{'='*60}")
  print(f"🧪 INICIANDO TEST LOCAL: {file_path}")
  print(f"{'='*60}")

  # 2. Instanciar DB Falsa
  mock_db = MockFirestoreClient()

  # 3. Detectar tipo y ejecutar (Router simplificado)
  name_upper = file_path.upper()
  
  if "FORM100" in name_upper:
    form_100.process_100(content, file_path, mock_db)
  if "FORM110" in name_upper:
    form_110.process_110(content, file_path, mock_db)
  elif "FORM400" in name_upper:
    form_400.process_400(content, file_path, mock_db)
  elif "FORM170" in name_upper:
    form_170.process_170(content, file_path, mock_db)
  elif "FORM500" in name_upper:
    form_500.process_500(content, file_path, mock_db)
  else:
    print("⚠️ No tengo configurado un test para este tipo de formulario en test_local.py")

In [7]:
x = "https://storage.googleapis.com/sicoescan/forms/21-0006-00-1149354-1-1_FORM110_1.html"
archivo_prueba = requests.get(x).text

run_local_test(archivo_prueba, x.split("/")[-1])


🧪 INICIANDO TEST LOCAL: 21-0006-00-1149354-1-1_FORM110_1.html
--- Procesando Formulario 110: 21-0006-00-1149354-1-1_FORM110_1.html ---
   [DB MOCK] Accediendo a colección: 'entidades'
   [DB MOCK] Accediendo a documento: '0006 - 00'
   [DB MOCK] Simulando .get()
✅ Formulario 110 procesado: 21-0006-00-1149354-1-1
